In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
import numpy as np
from exp_ntk_certify_label import run

In [5]:
seed = 4

data_params = dict(
    dataset = "csbm",
    learning_setting = "transductive", # or "transdructive"
    specification = dict(
        classes = 2,
        n_trn_labeled = 40, 
        n_trn_unlabeled = 0,
        n_val = 40,
        n_test = 120,
        sigma = 1,
        avg_within_class_degree = 1.58 * 2,
        avg_between_class_degree = 0.37 * 2,
        K = 1.5,
        seed = 4 # used to generate the dataset & data split
    )
)

model_params = dict(
    label = "APPNP",
    model = "APPNP",
    normalization = "sym_normalization",
    alpha = 0.25,
    iteration = 10,
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
    activation = "relu"
)

certificate_params = dict(
    n_adversarial = 1, # number adversarial nodes
    perturbation_model = "linf",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "cpu",
    dtype = torch.float64,
    allow_tf32 = False
)

In [6]:
data_params = dict(
    dataset = "us_county",    
    learning_setting = "transductive", # or "transdructive"
    cv_folds = 4,
    specification = dict(
        year=2012,    
        n_per_class = 50,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)
model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
    activation = "relu",
    solver = "qplayer",
    alpha_tol = 1e-4,
    bias = False,
)

In [21]:
A = np.ones((2,2))
z = np.array([[-2, 2]])
z_ = np.array([-2, 2]).reshape((2,1))
print(A*z)
print(A*z_)
print(A@z_)

[[-2.  2.]
 [-2.  2.]]
[[-2. -2.]
 [ 2.  2.]]
[[0.]
 [0.]]


In [25]:
#model_params["cache_size"] = 10000
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 10
certificate_params["delta"] = 0.1
certificate_params["delta_absolute"] = True
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
certificate_params["attack_nodes"] = "test"
other_params["path_gurobi_license"] = "/ceph/ssd/staff/gosl/app/gurobi.lic"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.1
model_params["solver"] = "qplayer"
model_params["alpha_tol"] = 1e-4
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-05-11 11:45:36 (INFO): Starting experiment exp_ntk_certify_label with configuration:
2024-05-11 11:45:36 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'transductive', 'specification': {'classes': 2, 'n_trn_labeled': 40, 'n_trn_unlabeled': 0, 'n_val': 40, 'n_test': 120, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 4}}
2024-05-11 11:45:36 (INFO): model_params: {'label': 'APPNP', 'model': 'APPNP', 'normalization': 'sym_normalization', 'alpha': 0.25, 'iteration': 10, 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer', 'alpha_tol': 0.0001, 'bias': False}
2024-05-11 11:45:36 (INFO): certification_params: {'n_adversarial': 10, 'perturbation_model': 'linf', 'delta': 0.1, 'delta_absolute': True, 'method': 'XXT', 'attack_nodes': 'test'}
2024-05-11 11:45:36 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-05-11 11:45:36 (INFO): other_params: {'device': 'cpu', 'dtype': torch.flo

CSBM mu:
[0.28347334 0.28347334 0.28347334 0.28347334 0.28347334 0.28347334
 0.28347334]
57 alphas found: ['0.0000', '0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '-0.0000', '0.0691', '0.1000', '0.1000', '0.0000', '0.1000', '0.1000', '0.0035', '0.1000', '0.1000', '0.0000', '0.0000', '0.1000', '0.0000', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.0161', '0.0671', '-0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.1000', '0.0020', '-0.0000', '0.1000', '0.0111', '0.0000', '0.1000', '0.1000', '0.0005', '0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '0.0000', '0.1000', '0.1000', '0.1000', '0.0133', '0.1000', '-0.0000', '0.0497', '0.1000', '0.1000', '-0.0000', '0.1000', '-0.0000', '-0.0000', '0.1000', '0.0977', '0.1000', '0.1000', '0.0004', '0.1000', '0.0364', '0.0135', '0.1000', '-0.0000', '-0.0000', '-0.0000', '-0.0000', '-0.0000', '0.1000']
Set parameter BestObjStop to value 0


/ceph/ssd/staff/gosl/src/ntk-robust/src/utils.py:983: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  m.addConstr(u <= M*s, "u_mil1")
2024-05-11 11:45:38 (INFO): Set parameter BestObjStop to value 0


Set parameter IntegralityFocus to value 1


2024-05-11 11:45:38 (INFO): Set parameter IntegralityFocus to value 1


...

2024-05-11 11:45:46 (INFO): Best objective 3.737613889189e+01, best bound 3.737613889189e+01, gap 0.0000%


2024-05-11 11:45:46 (INFO): 


User-callback calls 271, time in user-callback 0.00 sec


2024-05-11 11:45:46 (INFO): User-callback calls 271, time in user-callback 0.00 sec
2024-05-11 11:45:46 (INFO): Optimization status: 2
2024-05-11 11:45:46 (INFO): Original -0.83895, Opt objective 37.37614
2024-05-11 11:45:46 (INFO): Robust count 1 out of 4
2024-05-11 11:45:46 (INFO): Certified accuracy (poisoning): 0.008333333333333333


{'accuracy_test': 0.7833333611488342,
 'accuracy_trn': 0.9125000238418579,
 'accuracy_ub_test': 0.824999988079071,
 'accuracy_lb_test': 0.7166666388511658,
 'accuracy_ub_trn': 0.862500011920929,
 'accuracy_lb_trn': 0.887499988079071,
 'accuracy_cert_evasion_trivial': 0.45,
 'accuracy_cert_evasion_robust': 0.7416666746139526,
 'accuracy_cert_evasion_unrobust': 0.0,
 'accuracy_cert_pois_robust': 0.008333333333333333,
 'accuracy_cert_pois_unrobust': 0,
 'delta_absolute': 0.1,
 'y_true_cls': [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [32]:
model_params["cache_size"] = 10000
data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 0.01
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
certificate_params["cert_method"] = "ntk_bound"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.01
model_params["solver"] = "qplayer"
model_params["alpha_tol"] = 1e-8
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-12 14:55:17 (INFO): Starting experiment exp_ntk_cert_pois with configuration:
2024-04-12 14:55:17 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'transductive', 'specification': {'classes': 2, 'n_trn_labeled': 40, 'n_trn_unlabeled': 0, 'n_val': 40, 'n_test': 5, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 10}}
2024-04-12 14:55:17 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.01, 'pred_method': 'svm', 'activation': 'relu', 'cache_size': 10000, 'solver': 'qplayer', 'alpha_tol': 1e-08, 'bias': False}
2024-04-12 14:55:17 (INFO): certification_params: {'n_adversarial': 50, 'perturbation_model': 'linf', 'delta': 0.01, 'method': 'XXT', 'attack_nodes': 'all', 'cert_method': 'ntk_bound'}
2024-04-12 14:55:17 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-12 14:55:17 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf3

CSBM mu:
[0.375 0.375 0.375 0.375]
80 alphas found: ['0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100']
Set parameter BestObjStop to value 0


2024-04-12 14:55:17 (INFO): Set parameter BestObjStop to value 0


Set parameter IntegralityFocus to value 1


2024-04-12 14:55:17 (INFO): Set parameter IntegralityFocus to value 1


...

2024-04-12 14:55:19 (INFO): Set parameter IntegralityFocus to value 1


Set parameter IntFeasTol to value 0.0001


2024-04-12 14:55:19 (INFO): Set parameter IntFeasTol to value 0.0001
2024-04-12 14:55:20 (INFO): Original 0.03263, Opt objective -0.05662
2024-04-12 14:55:20 (INFO): Robust count 2 out of 5
2024-04-12 14:55:20 (INFO): Certified accuracy (poisoning): 0.4


Loaded user MIP start with objective 0.0326337



{'accuracy_test': 1.0,
 'accuracy_trn': 0.8125,
 'accuracy_ub_test': 0.800000011920929,
 'accuracy_lb_test': 0.6000000238418579,
 'accuracy_ub_trn': 0.862500011920929,
 'accuracy_lb_trn': 0.7250000238418579,
 'accuracy_cert_evasion_trivial': 0.0,
 'accuracy_cert_evasion_robust': 0.4000000059604645,
 'accuracy_cert_evasion_unrobust': 0.0,
 'accuracy_cert_pois_robust': 0.4,
 'accuracy_cert_pois_unrobust': 0,
 'y_true_cls': array([-1, -1,  1,  1,  1]),
 'y_pred_logit': array([-0.00811455, -0.13369031,  0.17732461,  0.06990783,  0.03263369]),
 'y_worst_obj': [0.08317413480441517,
  -0.044058926281888414,
  0.10940279116209606,
  -0.03322156070834449,
  -0.05661630497763761],
 'y_is_robust': [False, True, True, False, False],
 'y_opt_status': [2, 2, 2, 2, 2],
 'idx_train': array([ 9, 31, 38, 13, 29, 18, 30, 23, 36,  5,  4, 25, 41, 40, 27, 15, 32,
        33,  2,  7, 48, 44, 62, 83, 70, 82, 54, 47, 55, 77, 51, 66, 75, 60,
        49, 52, 58, 42, 63, 53]),
 'idx_val': array([ 6, 19, 12, 37, 1

In [31]:
model_params["cache_size"] = 10000
data_params["specification"]["avg_within_class_degree"] = 1.58 * 2
data_params["specification"]["avg_between_class_degree"] = 0.37 * 2
other_params["device"] = "cpu"
other_params["debug"] = False
certificate_params["n_adversarial"] = 50
certificate_params["delta"] = 0.01
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
certificate_params["cert_method"] = "ntk_bound"
verbosity_params["debug_lvl"] = "info"
model_params["regularizer"] = 0.01
model_params["solver"] = "qplayer"
model_params["alpha_tol"] = 1e-8
model_params["bias"] = False
data_params["learning_setting"] = "transductive"
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-04-11 16:19:07 (INFO): Starting experiment exp_ntk_certify with configuration:
2024-04-11 16:19:07 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'transductive', 'specification': {'classes': 2, 'n_trn_labeled': 40, 'n_trn_unlabeled': 0, 'n_val': 40, 'n_test': 120, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 10}}
2024-04-11 16:19:07 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.01, 'pred_method': 'svm', 'activation': 'relu', 'cache_size': 10000, 'solver': 'qplayer', 'alpha_tol': 1e-08, 'bias': False}
2024-04-11 16:19:07 (INFO): certification_params: {'n_adversarial': 50, 'perturbation_model': 'linf', 'delta': 0.01, 'method': 'XXT', 'cert_method': 'ntk_bound'}
2024-04-11 16:19:07 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-04-11 16:19:07 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 'debug': Fal

CSBM mu:
[0.28347334 0.28347334 0.28347334 0.28347334 0.28347334 0.28347334
 0.28347334]
79 alphas found: ['0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '-0.0000', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100', '0.0100']


{'accuracy': 0.9583333134651184,
 'accuracy_ub': 0.949999988079071,
 'accuracy_lb': 0.875,
 'accuracy_cert_trivial': 0.03333333333333333,
 'accuracy_cert': 0.8916666507720947,
 'accuracy_cert_unrobust': 0.0,
 'min_ypred': -0.8740060084083646,
 'max_ypred': 0.5618288466640802,
 'min_ylb': -0.9161557551719686,
 'max_ylb': 0.5366682086876243,
 'min_yub': -0.8320294475047423,
 'max_yub': 0.5874174960891775,
 'avg_ntklb': 0.7147890995803884,
 'min_ntklb': -0.8603377072824947,
 'max_ntklb': 18.429884713611802,
 'avg_ntkub': 0.8326720075041864,
 'min_ntkub': -0.8603377072824947,
 'max_ntkub': 18.429884713611802,
 'min_ntklabeled': -0.8603377072824947,
 'max_ntklabeled': 18.439884713611804,
 'avg_ntkunlabeled': 0.7765858675127242,
 'min_ntkunlabeled': -0.6694073080411328,
 'max_ntkunlabeled': 14.477873189149578,
 'cond': 5678.371688934855}